# Baselines and Formulation


In [ ]:
# Install and start postgresql-11 server
!sudo apt-get -y -qq update
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ `lsb_release -cs`-pgdg main" |sudo tee  /etc/apt/sources.list.d/pgdg.list
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql-11 postgresql-client-11
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `donors_choose` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS donors_choose;'

!sudo -u postgres psql -U postgres -c 'CREATE DATABASE donors_choose;'

# Environment variables for connecting to the database
%env DEMO_DATABASE_NAME=donors_choose
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432

# Setup a database with name `donors_choose` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS donors_choose;'

!sudo -u postgres psql -U postgres -c 'CREATE DATABASE donors_choose;'

# Environment variables for connecting to the database
%env DEMO_DATABASE_NAME=donors_choose
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432
%env DEMO_DATABASE_USER=postgres
%env DEMO_DATABASE_PASS=postgres

OK
deb http://apt.postgresql.org/pub/repos/apt/ focal-pgdg main
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 13.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 129496 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-136ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-136ubuntu1) ...
Selecting previously unselected package libjson-perl.
Preparing to unpack .../01-libjson-perl_4.02000-2_all.deb ...
Unpacking libjson-perl (4.02000-2) ...
Selecting previously unselected package netbase.
Preparing to unpack .../02-netbase_6.1_all.deb ...
Unpacking

In [ ]:
!pip install kaggle

!mkdir /root/.kaggle
!touch /root/.kaggle/kaggle.json
api_token = {"username":"ploped123","key":"eeeeba8fc52706723e4c1bcf41ae6fd3"}

import json
import zipfile
import os
import pandas as pd
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c kdd-cup-2014-predicting-excitement-at-donors-choose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 99% 917M/926M [00:06<00:00, 209MB/s]
100% 926M/926M [00:06<00:00, 148MB/s]


In [ ]:
!ls /content

kdd-cup-2014-predicting-excitement-at-donors-choose.zip  sample_data


In [ ]:
!unzip /content/kdd-cup-2014-predicting-excitement-at-donors-choose.zip

Archive:  /content/kdd-cup-2014-predicting-excitement-at-donors-choose.zip
  inflating: donations.csv.zip       
  inflating: essays.csv.zip          
  inflating: outcomes.csv.zip        
  inflating: projects.csv.zip        
  inflating: resources.csv.zip       
  inflating: sampleSubmission.csv.zip  


In [ ]:
!unzip /content/donations.csv.zip

Archive:  /content/donations.csv.zip
  inflating: donations.csv           


In [ ]:
!unzip /content/projects.csv.zip

Archive:  /content/projects.csv.zip
  inflating: projects.csv            


In [ ]:
full_donations = pd.read_csv("/content/donations.csv")
full_projects = pd.read_csv("/content/projects.csv")

In [ ]:
from datetime import timedelta

## Calculating base rate on full dataset

In [ ]:
a = pd.merge(full_projects, full_donations, on=['projectid'], how='left')
a['in_4_months'] = (pd.to_datetime(a['donation_timestamp']) - pd.to_datetime(a['date_posted'])) < timedelta(days=120)
values = {'donation_to_project': 0.0}
a = a.fillna(value=values)
a.loc[a['in_4_months'] == False, 'donation_to_project'] = 0.0
donation_in_4_months = a.groupby(['projectid'])['donation_to_project'].sum().reset_index(name='donation_in_4_months')
df = pd.merge(full_projects, donation_in_4_months, on=['projectid'], how='left')


In [ ]:
df['if_fully_funded_after_4_months'] = df['total_price_excluding_optional_support'] <= df['donation_in_4_months']
df['if_fully_funded_after_4_months'].value_counts()
print("Baserate % projects fully funded:", df['if_fully_funded_after_4_months'].mean())

Baserate % projects fully funded: 0.5480215269433126


In [ ]:
df.head()

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,donation_in_4_months,if_fully_funded_after_4_months
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.576340,-119.608713,Selma,CA,93662.0,NaN,...,Grades 6-8,30.0,555.81,653.89,32.0,f,f,2014-05-12,0.0,False
1,90de744e368a7e4883223ca49318ae30,864eb466462bf704bf7a16a585ef296a,d711e47810900c96f26a5d0be30c446d,4.837020e+11,32.911179,-96.723640,Dallas,TX,75243.0,urban,...,Grades PreK-2,30.0,296.47,348.79,22.0,f,f,2014-05-12,0.0,False
2,32943bb1063267de6ed19fc0ceb4b9a7,37f85135259ece793213aca9d8765542,665c3613013ba0a66e3a2a26b89f1b68,4.103270e+11,45.166039,-122.414576,Colton,OR,97017.0,rural,...,Grades PreK-2,30.0,430.89,506.93,17.0,f,f,2014-05-11,0.0,False
3,bb18f409abda2f264d5acda8cab577a9,2133fc46f951f1e7d60645b0f9e48a6c,4f12c3fa0c1cce823c7ba1df57e90ccb,3.600153e+11,40.641727,-73.965655,Brooklyn,NY,11226.0,urban,...,Grades 3-5,30.0,576.07,677.73,12.0,f,f,2014-05-11,0.0,False
4,24761b686e18e5eace634607acbcc19f,867ff478a63f5457eaf41049536c47cd,10179fd362d7b8cf0e89baa1ca3025bb,6.227100e+10,34.043939,-118.288371,Los Angeles,CA,90006.0,urban,...,Grades PreK-2,30.0,408.40,480.47,24.0,f,f,2014-05-11,0.0,False


In [ ]:
pd.to_datetime(df['date_posted']).describe()

<ipython-input-14-e3aa2acbd2c1>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  pd.to_datetime(df['date_posted']).describe()


count                  664098
unique                   4026
top       2012-09-30 00:00:00
freq                     1490
first     2002-09-13 00:00:00
last      2014-05-12 00:00:00
Name: date_posted, dtype: object

In [ ]:
pd.to_datetime(df['date_posted']).sort_values().iloc[0]

Timestamp('2002-09-13 00:00:00')

In [ ]:
from datetime import datetime

In [ ]:
datetime.now() + relativedelta(months=6)

datetime.datetime(2023, 8, 2, 21, 14, 51, 614681)

relativedelta(months=+6)

In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
def chron_split_date_ranges(dates, train_length = relativedelta(years=1), val_length = relativedelta(months=1), test_length = relativedelta(months=1), retrain_length = relativedelta(weeks=2), min_f_col = relativedelta(months=1), min_l_col = relativedelta(months=4)):
  sorted = dates.sort_values()
  first_date = sorted.iloc[0]
  last_date = sorted.iloc[-1]

  split_dates = []

  train_start = first_date + min_f_col

  while train_start < last_date - val_length - train_length - 2 * min_l_col:
    #print(train_start)
    val_start = train_start + train_length + min_l_col
    test_start = val_start + test_length
    split_dates.append((train_start, val_start, test_start, train_start + min_l_col, val_start + min_l_col, test_start + min_l_col))
    train_start = train_start + retrain_length
  return split_dates

In [ ]:
chron_split_date_ranges = chron_split_date_ranges(pd.to_datetime(df['date_posted'])[::10])

In [ ]:
chron_split_date_ranges

[(Timestamp('2002-10-17 00:00:00'),
  Timestamp('2004-02-17 00:00:00'),
  Timestamp('2004-03-17 00:00:00'),
  Timestamp('2003-02-17 00:00:00'),
  Timestamp('2004-06-17 00:00:00'),
  Timestamp('2004-07-17 00:00:00')),
 (Timestamp('2002-10-31 00:00:00'),
  Timestamp('2004-02-29 00:00:00'),
  Timestamp('2004-03-29 00:00:00'),
  Timestamp('2003-02-28 00:00:00'),
  Timestamp('2004-06-29 00:00:00'),
  Timestamp('2004-07-29 00:00:00')),
 (Timestamp('2002-11-14 00:00:00'),
  Timestamp('2004-03-14 00:00:00'),
  Timestamp('2004-04-14 00:00:00'),
  Timestamp('2003-03-14 00:00:00'),
  Timestamp('2004-07-14 00:00:00'),
  Timestamp('2004-08-14 00:00:00')),
 (Timestamp('2002-11-28 00:00:00'),
  Timestamp('2004-03-28 00:00:00'),
  Timestamp('2004-04-28 00:00:00'),
  Timestamp('2003-03-28 00:00:00'),
  Timestamp('2004-07-28 00:00:00'),
  Timestamp('2004-08-28 00:00:00')),
 (Timestamp('2002-12-12 00:00:00'),
  Timestamp('2004-04-12 00:00:00'),
  Timestamp('2004-05-12 00:00:00'),
  Timestamp('2003-04-12 

In [ ]:
df['date_posted'].describe()

# Common Sense analysis on demo data

In [ ]:
!curl -s -OL https://dsapp-public-data-migrated.s3.us-west-2.amazonaws.com/donors_sampled_20210920_v3.dmp
!PGPASSWORD=$DEMO_DATABASE_PASS pg_restore -h $DEMO_DATABASE_HOST -p $DEMO_DATABASE_PORT -d $DEMO_DATABASE_NAME -U $DEMO_DATABASE_USER -O -j 8 donors_sampled_20210920_v3.dmp

In [ ]:
!pip install SQLAlchemy==1.3.18 PyYAML==6.0 psycopg2-binary==2.9.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.4.46
    Uninstalling SQLAlchemy-1.4.46:
      Successfully uninstalled SQLAlchemy-1.4.46


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

db_url = URL(
            'postgres',
            host='localhost',
            username='postgres',
            database='donors_choose',
            password='postgres',
            port=5432,
        )

db_engine = create_engine(db_url)

In [ ]:
outcomes_df = pd.read_sql("SELECT * FROM data.outcomes", db_engine)
donations_df = pd.read_sql("SELECT * FROM data.donations", db_engine)
resources_df = pd.read_sql("SELECT * FROM data.resources", db_engine)

In [ ]:
essays_df = pd.read_sql("SELECT * FROM data.essays", db_engine)
ts_df = pd.read_sql("SELECT * FROM data.time_series_features", db_engine)

In [ ]:
projects_df.head()

,schoolid,projectid_str,teacher_acctid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,teacher_ny_teaching_fellow,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_asking_price,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,entity_id
0,29a2da790e38b6c8a1c70aced6b9c765,30c034618e67d00c641f9b5b7775c0f4,0903da60e148adc6280d55f5d94791a5,192013001182,41.428391,-91.049135,Muscatine,IA,52761,None,Muscatine Cmty School District,Muscatine,False,False,False,False,False,False,Ms.,False,False,Applied Sciences,Math & Science,College & Career Prep,Applied Learning,Supplies,high poverty,Grades 6-8,35.0,511.32,601.55,100,False,False,2012-08-06,234148
1,06ef48acbdf9b013d4bc4bfc8d328cc1,94199c544d9d2926c1820e5c6cde1eb6,fb340c7ac3b22a5984c6a82602e4a510,450111000143,32.233070,-80.855905,Bluffton,SC,29910,rural,Beaufort Co School District,Beaufort,False,False,False,False,False,False,Ms.,False,False,Literacy,Literacy & Language,ESL,Literacy & Language,Supplies,high poverty,Grades 3-5,35.0,167.43,192.45,40,False,False,2010-07-10,453579
2,06ef48acbdf9b013d4bc4bfc8d328cc1,61f42f174afef1ed2419ab948a647137,fb340c7ac3b22a5984c6a82602e4a510,450111000143,32.233070,-80.855905,Bluffton,SC,29910,rural,Beaufort Co School District,Beaufort,False,False,False,False,False,False,Ms.,False,False,Literacy,Literacy & Language,Applied Sciences,Math & Science,Other,high poverty,Grades 3-5,35.0,167.43,196.98,45,False,False,2011-06-09,353855
3,06ef48acbdf9b013d4bc4bfc8d328cc1,c966f5226f42aaaf6b115d7dbaefdea3,e0b5a72f41a376b28db9c2e838a24de5,450111000143,32.233070,-80.855905,Bluffton,SC,29910,rural,Beaufort Co School District,Beaufort,False,False,False,False,False,False,Mrs.,False,False,Literacy,Literacy & Language,Health & Life Science,Math & Science,Other,high poverty,Grades 3-5,35.0,162.14,190.75,25,False,False,2011-06-13,353178
4,06ef48acbdf9b013d4bc4bfc8d328cc1,69e9c24111daa1e6ba64c6d5538e4df1,32e86c49396707f71fdc0398ab2b844b,450111000143,32.233070,-80.855905,Bluffton,SC,29910,rural,Beaufort Co School District,Beaufort,False,False,False,False,False,False,Mrs.,False,False,History & Geography,History & Civics,Environmental Science,Math & Science,Books,high poverty,Grades 3-5,35.0,381.27,448.55,25,False,False,2012-07-15,239363


In [ ]:
donations_df.head()

,donationid,projectid_str,donor_acctid,donor_city,donor_state,donor_zip,is_teacher_acct,donation_timestamp,donation_to_project,donation_optional_support,donation_total,dollar_amount,donation_included_optional_support,payment_method,payment_included_acct_creadit,payment_included_campaign_gift_card,payment_included_web_purchased_gift_card,payment_was_promo_matched,via_giving_page,for_honoree,donation_message,entity_id
0,705ef855c2d90eed594cf26e634add93,fb9d9bebbd2309cbd948a03c00ec65b0,112631d2731d90f8ac0ae542d67904fe,,None,None,False,2009-11-09 10:49:34.648,41.00,9.00,50.0,10_to_100,True,no_cash_received,False,True,False,False,False,False,None,495172
1,59b71f584f3516f75d4e98d8acbaa65f,f93456e60338550f3ad7a2a0e87617dd,127936d3a07b33d3012bb23a9464532a,Austin,TX,78704,False,2011-09-02 08:48:01.447,85.00,15.00,100.0,100_and_up,True,paypal,False,False,False,False,True,False,None,336273
2,8bcc055f586e0343dd8779fdd8ac4775,f7848f096af41049e8dc6af81139d2b6,55223d1a24127d2fcce3a0b45e917b03,Chicago,IL,60093,False,2013-12-01 09:49:55.145,17.00,3.00,20.0,10_to_100,True,amazon,False,False,False,False,False,False,None,99049
3,3afbbc4257049362dab620251c49a4a0,f3d5ec1500fe57584856d4cce2d66161,88c6f8743b0340853b548404f8de9e5b,None,PR,None,False,2013-05-13 09:41:48.006,4.25,0.75,5.0,under_10,True,no_cash_received,False,True,False,False,False,False,Cause,173719
4,acf6e26e501def0b285834026459e592,f33b72a6093a64696d3a8e7ef2d0afa7,85e51d426e69e46b785b0225117b7e8b,None,None,None,True,2011-01-11 18:58:57.280,8.50,1.50,10.0,10_to_100,True,creditcard,False,False,False,False,True,False,None,395246


In [ ]:
resources_df.head()

,resourceid,projectid_str,vendorid,vendor_name,project_resource_type,item_name,item_number,item_unit_price,item_quantity,entity_id,date_posted
0,202d2fe1164630c3e18623345650175d,0000fa3aa8f6649abab23615b546016d,7,AKJ Books,Books,Soft Rain,9780440412427,4.43,30.0,255654,2012-04-07
1,f0fa19ae0953d19e8ec670309fc8da72,071c87944d727b6dff382271125ff669,None,None,Trips,None,None,NaN,NaN,218307,2012-09-14
2,7ac0367031ced1660b7ce7bc903a29b1,07b961fee8623e35526bc6168d2911e9,7,AKJ Books,Books,The Purple Balloon,9780375841460,13.25,1.0,376890,2011-03-06
3,40d597c5b2889f95e37ad3d795dece16,0859ba7d65068d8028b7b8a06c6a3f8c,None,None,Trips,None,None,NaN,NaN,604216,2007-11-08
4,51e72dfc8c2b4c1a51045b2c3ccd5579,0aca0bb0e24f7b3731ce6af5321628d7,7,AKJ Books,Supplies,Graffiti Women,9780810957473,25.55,1.0,88543,2013-10-03


In [ ]:
projects_df['date_posted'].describe()

<ipython-input-43-8b090f1a4ccb>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  projects_df['date_posted'].describe()


count                   16480
unique                   2815
top       2014-02-23 00:00:00
freq                       53
first     2002-09-19 00:00:00
last      2014-05-11 00:00:00
Name: date_posted, dtype: object

We have data spanning around 12 years, so we can use feature and label constructions that require several months of feature and label availability and only lose a small portion of the data.

In [ ]:
from datetime import timedelta
import numpy as np

In [ ]:
project_donations = {}
project_requests = {}
project_postedts = {}
for i in range(len(projects_df)):
  project_donations[projects_df.iloc[i]['projectid_str']] = 0
  project_requests[projects_df.iloc[i]['projectid_str']] = projects_df.iloc[i]['total_asking_price'] 
  project_postedts[projects_df.iloc[i]['projectid_str']] = projects_df.iloc[i]['date_posted']

for i in range(len(donations_df)):
  project = donations_df.iloc[i]['projectid_str']
  donation_ts = donations_df.iloc[i]['donation_timestamp']
  posted_ts = project_postedts[project]
  if donation_ts - posted_ts > timedelta(days = 120):
    continue
  project_donations[project] += donations_df.iloc[i]['donation_to_project']

In [ ]:
project_funded = {}
for project in project_requests:
  if project_donations[project] >= project_requests[project]:
    project_funded[project] = 1
  else:
    project_funded[project] = 0

In [ ]:
essays_df['essay_length'] = essays_df['essay'].str.len()
essays_df['need_length'] = essays_df['need_statement'].str.len()
essays_df['title_length'] = essays_df['title'].str.len()

In [ ]:
labeled_df = projects_df.merge(pd.DataFrame.from_dict({'projectid_str': project_funded.keys(), 'fully_funded_4mo': project_funded.values()}), on=['projectid_str'])
labeled_df = labeled_df.merge(essays_df, on = ['projectid_str'], how = 'inner').dropna(subset = ['need_length'])

In [ ]:
labeled_df['need_length'].describe()

count    16474.000000
mean       123.577941
std         64.897064
min         23.000000
25%         82.000000
50%        113.000000
75%        153.000000
max       1124.000000
Name: need_length, dtype: float64

In [ ]:
# Baseline Probability of a project being fully funded in 4 months
labeled_df['fully_funded_4mo'].mean()

0.5295010319291004

In [ ]:
# Projects farther North are more likely to be funded
np.corrcoef(labeled_df['school_latitude'], labeled_df['fully_funded_4mo'])[0,1]

0.03547896838427393

In [ ]:
# Projects farther East are more likely to be funded
np.corrcoef(labeled_df['school_longitude'], labeled_df['fully_funded_4mo'])[0,1]

-0.05465138145314809

In [ ]:
# Projects with smaller requested amounts are more likely to be funded
np.corrcoef(labeled_df['total_asking_price'], labeled_df['fully_funded_4mo'])[0,1]

-0.11264815601741877

In [ ]:
# Weak correlation between number of students reached and probability of funding
np.corrcoef(labeled_df['students_reached'], labeled_df['fully_funded_4mo'])[0,1]

-0.008562035307620421

In [ ]:
# Longer titles positively correlated with full funding
np.corrcoef(labeled_df['title_length'], labeled_df['fully_funded_4mo'])[0,1]

0.0381443858355917

In [ ]:
np.corrcoef(labeled_df['need_length'], labeled_df['fully_funded_4mo'])[0,1]

0.027217687745323516

In [ ]:
np.corrcoef(labeled_df['essay_length'], labeled_df['fully_funded_4mo'])[0,1]

0.009384998757455482

Many features of interest are categorical in nature and computing correlations between these features and the binary target isn't trivial.  To determine potentially informative categorical features, we can one hot encode the categories and calculate the empirical probability of funding given a sample in that category.  Variables where different categories have dramatically different empirical probabilities are most likely to be predictive, which we can quantify with mean absolute devation of each category's label mean.

In [ ]:
labeled_df['poverty_level'].unique()

array(['high poverty', 'highest poverty', 'moderate poverty',
       'low poverty'], dtype=object)

In [ ]:
labeled_df.groupby('poverty_level').mean()['fully_funded_4mo']

poverty_level
high poverty        0.494591
highest poverty     0.560631
low poverty         0.514045
moderate poverty    0.458333
Name: fully_funded_4mo, dtype: float64

In [ ]:
labeled_df.groupby('grade_level').mean()['fully_funded_4mo']

grade_level
Grades 3-5       0.513767
Grades 6-8       0.543106
Grades 9-12      0.568407
Grades PreK-2    0.514017
Name: fully_funded_4mo, dtype: float64

In [ ]:
labeled_df.groupby('teacher_prefix').mean()['fully_funded_4mo']

teacher_prefix
Mr.     0.561348
Mrs.    0.496842
Ms.     0.557627
Name: fully_funded_4mo, dtype: float64

In [ ]:
labeled_df.groupby('primary_focus_subject').mean()['fully_funded_4mo']

primary_focus_subject
Applied Sciences         0.551122
Character Education      0.590308
Civics & Government      0.792453
College & Career Prep    0.455128
Community Service        0.690476
ESL                      0.486726
Early Development        0.449180
Economics                0.600000
Environmental Science    0.573574
Extracurricular          0.412698
Foreign Languages        0.425373
Gym & Fitness            0.445946
Health & Life Science    0.571918
Health & Wellness        0.489933
History & Geography      0.572301
Literacy                 0.524821
Literature & Writing     0.543344
Mathematics              0.520803
Music                    0.611111
Nutrition                0.461538
Other                    0.399351
Parent Involvement       0.468750
Performing Arts          0.546341
Social Sciences          0.487903
Special Needs            0.477426
Sports                   0.666667
Visual Arts              0.542029
Name: fully_funded_4mo, dtype: float64

In [ ]:
labeled_df.groupby('primary_focus_area').mean()['fully_funded_4mo']

primary_focus_area
Applied Learning       0.472198
Health & Sports        0.510490
History & Civics       0.562724
Literacy & Language    0.527152
Math & Science         0.542361
Music & The Arts       0.571148
Special Needs          0.477426
Name: fully_funded_4mo, dtype: float64

In [ ]:
labeled_df.groupby('resource_type').mean()['fully_funded_4mo']

resource_type
Books         0.604280
Other         0.516435
Supplies      0.553119
Technology    0.455551
Trips         0.614583
Visitors      0.434783
Name: fully_funded_4mo, dtype: float64

# QUESTION 2

For this question, you'll develop a model to help DonorsChoose **identify 10% of projects with the highest risk of failing to meet their funding goal** for their digital content expert review and provide suggestions. In order to intervene early in the process, DonorsChoose wants to identify these projects to help **immediately upon being posted** to the site.

Build and evaluate the performance of several machine learning models for this task. **Be sure to use comments or text blocks to discuss the choices and assumptions you're making along the way.** Feel free to use any python packages available (such as sklearn) for this.

In order to fairly evaluate models on this problem, we'll need to chronologically split the data into training, validation, and test sets. We make model decisions and select hyperparameters on a validation set in order to account for overfitting, then evaluate on a test set to account for selection bias in estimating our model's performance on future, unseen data.  

The chronological split is necessary because our model will ultimately need to perform on projects farther into the future; it is possible that using older data in validation and test sets may constitute cheating and misrepresent a model's performance on future data.

In using test performance as an estimate for future performance, we assume little or no distribution shift between our dataset and future projects in the real world.  This is unrealistic because funding behaviors by donators may change over time, as will the distribution of project features, descriptions, and legitimacy.  Furthermore, there may be differences between the collection of the original dataset and sampling live data in production which could result in covariate shift or data discrepancies.

In [ ]:
labeled_df = labeled_df.sort_values('date_posted_x').reset_index(drop = True)

In [ ]:
num_cols = ['school_latitude', 'school_longitude', 'total_asking_price', 'title_length', 'need_length']
cat_cols = ['poverty_level', 'grade_level', 'teacher_prefix']

In [ ]:
X = labeled_df[num_cols].join(pd.get_dummies(labeled_df[cat_cols]))
train_X = X.iloc[:int(len(X) * .6)]
X -= train_X.mean()
X /= train_X.std()
Y = labeled_df['fully_funded_4mo']

In [ ]:
train_X = X.iloc[:int(len(X) * .6)]
val_X = X.iloc[int(len(X) * .6): int(len(X) * .8)]
test_X = X.iloc[int(len(X) * .8):]

train_Y = Y.iloc[:int(len(X) * .6)]
val_Y = Y.iloc[int(len(X) * .6): int(len(X) * .8)]
test_Y = Y.iloc[int(len(X) * .8):]

In [ ]:
# TPR Baselines for each split
print(1 - np.mean(train_Y), 1 - np.mean(val_Y), 1 - np.mean(test_Y))

0.4637798462161068 0.3766312594840667 0.5845220030349014


Based on the wording of the task, it makes most sense to approach this problem with binary classification of whether or not a given project was fully funded within 4 month.  An alternative approach would be to train a regression predicting the dollar amount of donations received, or the numerical ratio of donations received to requested amount.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def top_k_tpr(pred, vals, k = .1):
  return 1 - np.mean(vals.iloc[np.argsort(pred)].values[:int(k*len(vals))])

def evaluate_model(model):
  model.fit(train_X, train_Y)
  train_pred = model.predict(train_X)
  val_pred = model.predict(val_X)
  test_pred = model.predict(test_X)
  print(f"Train corr: {np.corrcoef(train_pred, train_Y)[0,1]}\nTrain top 10% TPR: {top_k_tpr(train_pred, train_Y)}\nVal corr: {np.corrcoef(val_pred, val_Y)[0,1]}\nVal top 10% TPR: {top_k_tpr(val_pred, val_Y)}\nTest corr: {np.corrcoef(test_pred, test_Y)[0,1]}\nTest top 10% TPR: {top_k_tpr(test_pred, test_Y)}")

In [ ]:
logreg = LogisticRegression(max_iter = 1000)
evaluate_model(logreg)

Train corr: 0.24892472690790474
Train top 10% TPR: 0.6366396761133604
Val corr: 0.10927693517264568
Val top 10% TPR: 0.452887537993921
Test corr: 0.06958675915275872
Test top 10% TPR: 0.541033434650456


In [ ]:
rfc = RandomForestClassifier()
evaluate_model(rfc)

Train corr: 0.9993899690454462
Train top 10% TPR: 1.0
Val corr: 0.16692597207749718
Val top 10% TPR: 0.5197568389057751
Test corr: 0.07700085419057597
Test top 10% TPR: 0.6413373860182371


In [ ]:
nn = KNeighborsClassifier(n_neighbors=21)
evaluate_model(nn)

Train corr: 0.29455486321522456
Train top 10% TPR: 0.638663967611336
Val corr: 0.06329253025113005
Val top 10% TPR: 0.4042553191489362
Test corr: 0.05552173503329483
Test top 10% TPR: 0.5835866261398177


# QUESTION 3

Briefly discuss (in at most 1-2 paragraphs) the results of your analysis above: Which model would you select to deploy and why? Are there future analysis or improvements you would suggest? Are there other data sources you would ideally like to be able to incorporate into your models?

Of the three candidate model classes, the random forest performed best in both of validation correlation and true positive rate of the top 10% most predicted at risk.  The random forest further performed the best on the test set as well, suggesting that it is most successful in capturing signals which generalize to unseen data.  Moreover, it outperforms a baseline TPR of the overall rate of failing to receive full funding.

Future work should include a more complete ablation of features.  In this assignment, I excluded categorical features with many values because they would dramatically increase the dimension of the feature space, making models more prone to overfitting.  This could be addressed by grouping certain categories together, or, in the extreme case, using one vs. the rest.  I'm also interested in analyzing the text data beyond just length via sentiment analysis or pretrained embeddings.

# Submission

To submit your exercise, please **save a copy** of this notebook containing your code and outputs (you can save it either to google drive or github, but make sure the course staff will have read permissions to access it).

Provide a link to your copy of the notebook when you submit the assignment in canvas.

Thank you and we're looking forward to a great semester in 10718!